SIMPLE NEURAL NETWORK IN PYTORCH

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [53]:
device=torch.device('cpu')

**torch.nn**

Neural networks can be constructed using the torch.nn package.<br>

Provides pretty much all neural network related functionalities such as :<br>

Linear layers - nn.Linear, nn.Bilinear <br>
Convolution Layers - nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.ConvTranspose2d<br>
Nonlinearities - nn.Sigmoid, nn.Tanh, nn.ReLU, nn.LeakyReLU<br>
Pooling Layers - nn.MaxPool1d, nn.AveragePool2d<br>
Recurrent Networks - nn.LSTM, nn.GRU<br>
Normalization - nn.BatchNorm2d<br>
Dropout - nn.Dropout, nn.Dropout2d<br>
Embedding - nn.Embedding<br>
Loss Functions - nn.MSELoss, nn.CrossEntropyLoss, nn.NLLLoss<br>

model class

In [54]:
class NN(nn.Module):
    def __init__(self , input_size, num_classes):   #28X28 pixel size   and total 10 classes
        super(NN,self).__init__()   
        self.fc1= nn.Linear(input_size, 50)
        self.fc2=nn.Linear(50, num_classes)

    
    def forward(self , x):
        x=self.fc1(x)
        x=F.relu(x)    #activation function
        x= self.fc2(x)
        return x

In [55]:
#checking
model=NN(784, 10)           #784 = 28X28 pixel (input size)  and 10 is the number of classes (output size)
x =torch.randn(64,784)      #we are creating a 64x784 which means 64 examples of 784 pixels (64 random training examples)
model(x).shape              #predicting 

torch.Size([64, 10])

meta-data

In [56]:
input_size=784
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=5

data loaders

In [57]:
#totransforms will convert the PIL image or the ndarray to tensors
#train_dataset will download the dataset
#train_loader and test_loader will load the dataset
#dataloader will load the dataset in batches of batchsize
# shuffle=True will jumble all the images in the dataset so that the model is not biased towards any particular image
#shuffle =True will make sure that the same images are present in a a batch in a different epoch
#root='dataset/' is the where the images to be stored

train_dataset= datasets.MNIST(root='dataset/', train=True , transform=transforms.ToTensor() , download=True)
train_loader=DataLoader(dataset=train_dataset , batch_size=batch_size , shuffle= True)
test_dataset=datasets.MNIST(root='dataset/', train=False , transform=transforms.ToTensor() , download=True)
test_loader=DataLoader(dataset=test_dataset , batch_size=batch_size , shuffle= False)


initialize loss and optim function

In [58]:
#initialize
model=NN(input_size=input_size , num_classes=num_classes).to(device)

criterion= nn.CrossEntropyLoss() #loss function (log loss) multi class
optimizer=optim.Adam(model.parameters() , lr=learning_rate)      #requires model parameters 
#Adam is faster to converge. SGD is slower but generalizes better. use accor to situation.

#learning rate is the rate at which the model learns from the data
#If it is too high it means that the model will learn too fast and will not be able to find the optimal solution
#If it is too low, the model will learn too slow and will not be able to find the optimal solution.  so find balance

model training

In [59]:
#mnist dataset has 939X64 training examples(60k)
#batch_idx is the batch num
#64X1X28X28     28X28 image size(pixels)  , 1 is for grayscale image (no rgb)  , 64 is batch size
#64X1X28X28 we convert this to 64 X 784 (1X28X28)  we flattened 1X28X28 into single dim and we have 64 training ex

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data=data.to(device=device)
        targets=targets.to(device=device)
        
        #reshape
        data=data.reshape(data.shape[0],-1)

        #forward
        outputs=model(data)                #produces outputs , out.shape = [64,10] , for each training ex give 10 values(probs of 10 classes)
        loss=criterion(outputs,targets)    #loss func , find loss btw models predictions (outputs) and true labels(targets)
        
        #backward
        optimizer.zero_grad()              #Before backpropagation, you need to zero out (reset) the gradients of all the parameters.
        loss.backward()                    #the gradients of the loss are computed
        
        #gradient descent or adam step
        optimizer.step()                   #the optimizer adjusts the weights and biases (i.e., the parameters) based on computed gradients


accuracy 

In [60]:
#checks the accuarcy of a model on a given dataset

#model.eval - This line puts the model in evaluation mode, which turns off certain behaviors specific to training, such as dropout and 
#batch normalization updates. In evaluation mode, the model performs inference without changing internal states like in training.


def check_accuracy(loader, model):
    if loader.dataset.train:
        print('checking accuracy on training data')
    else:
        print('checking accuracy on test data')

    num_correct=0
    num_samples=0 
    model.eval()   #evaluation mode
  
    with torch.no_grad():           #torch.no_grad() is a context manager that disables gradient calculation
        for x, y in loader: 
            x=x.to(device=device)
            y=y.to(device=device)
            x=x.reshape(x.shape[0],-1)

            scores=model(x)         #outputs [64,10]

            _, predictions = scores.max(dim=1)     
            #max at each row (so total 64 rows and 10 cols and we choose 1 value at each row )
            # '_' (first returned val) returns the values (max values of logits or probabilities) 
            # 'predictions' (Second returned val) returns the indices(max val indices) 
                                                
            num_correct += (predictions==y).sum()    #.sum() counts how many predictions are correct in the current batch.
            num_samples += predictions.size(0)       #keeps track of the total number of samples processed.


        print(f'got {num_correct} / {num_samples} with acuuracy {float(num_correct) / float(num_samples) * 100:.2f}')

    model.train()   #sets the model back to training mode from eval mode


In [61]:
check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

checking accuracy on training data
got 58318 / 60000 with acuuracy 97.20
checking accuracy on test data
got 9663 / 10000 with acuuracy 96.63
